In [ ]:
using FFTW
using Plots

In [ ]:
L = 50
N = 500
dx = L/N
dk = 2π/L
dt = 5e-3

In [ ]:
x1 = dx .* collect(0:N-1)
x2 = dx .* collect(0:N-1)'
k1 = rfftfreq(N , N * dk)
k2 = fftfreq(N, N * dk)';

In [ ]:
k² = @. (k1^2 + k2^2);
k = sqrt.(k²)
c = - k²
f1 = exp.(c .*dt)
f2 = (f1 .- 1.) ./ c
f2[1, 1] = 0

In [ ]:
F = plan_rfft(x1 .+ x2)
B = plan_irfft(im*k, N);

In [ ]:
kmax = maximum(k)
twothirds = k .>= kmax * 2/3
function antialias!( F )
    F[twothirds] .= 0.
end

In [ ]:
V = @. cos(dk * x1) * cos(dk * x2) 
FV = F*V
Fv = @. - k² * FV
v = B*Fv

function etd!(Fφ, φ)
    Fvφ = F*(v.*φ)
    @. Fφ = f1 * Fφ
    @. Fφ += (f2 * Fvφ)
    φ .= B*Fφ
    antialias!(Fφ)
end

In [ ]:
dφ = zeros((N, N))
φ = zeros((N, N))
φ[1:N÷2,1:N÷2] .= 1.
# φ = @. exp(- ((x1-L/2)^2 + (x2-L/2)^2) / 2)
Fφ = F*φ
plot(x1, x2, φ; st=:surface, zrange=(-.1, 1.1))

In [ ]:
plot(x1, x2, V; st=:surface)

In [ ]:
M = 100
for t in 1:M
    etd!(Fφ, φ)
end
print(maximum(abs.(φ - φ')))

plot(; zrange=(-1, 1.1))
plot!(x1, x2, φ; st=:surface)

In [ ]:
M = 10_000

dφ = zeros((N, N))
φ = zeros((N, N))
φ[1:N÷2, 1:N÷2] .= 1.
Fφ = F*φ

frames = 1000
n = M÷frames
φt = zeros((frames, N, N))
φt[1,:,:] = φ

for i in n:M
    etd!(Fφ,φ)
    if i%n==n-1
        φt[i÷n+1,:, :] = φ
    end
end

In [ ]:
anim = @animate for i in 1:frames
    plot(x1, x2, φt[i,:,:]; st=:surface, zrange=(-.1, 1.5))
    plot!(size=(1200, 1200))
end every 1;
gif(anim, "diff_2D.mp4", fps = 20)

In [ ]:
print(maximum(abs.(φ - φ')))